In [ ]:
import os
import torch
import torch.nn.functional as F
import numpy as np
import cv2

import time
import math

import argparse

import dsacstar
from network import Network
import datasets
from utils import tr, reverse_tr
import tensorboard


In [2]:
def compute_ABC(w_t_c, c_R_w, w_t_chat, chat_R_w, c_n, eye):
    """
    Computes A, B, and C matrix given estimated and ground truth poses
    and normal vector n.
    `w_t_c` and `w_t_chat` must have shape (batch_size, 3, 1).
    `c_R_w` and `chat_R_w` must have shape (batch_size, 3, 3).
    `n` must have shape (3, 1).
    `eye` is the (3, 3) identity matrix on the proper device.
    """
    chat_t_c = chat_R_w @ (w_t_c - w_t_chat)
#     print(f"in abc chatRW={chat_R_w.shape} and transpose={c_R_w.transpose(1,2).shape}")
    chat_R_c = chat_R_w @ c_R_w.transpose(1, 2)

    A = eye - chat_R_c
    C = c_n @ chat_t_c.transpose(1, 2)
    B = C @ A
    A = A @ A.transpose(1, 2)
    B = B + B.transpose(1, 2)
    C = C @ C.transpose(1, 2)

    return A, B, C


class LocalHomographyLoss(torch.nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()

        # `c_n` is the normal vector of the plane inducing the homographies in the ground-truth camera frame
        self.c_n = torch.tensor([0, 0, -1], dtype=torch.float32, device=device).view(3, 1)

        # `eye` is the (3, 3) identity matrix
        self.eye = torch.eye(3, device=device)

    def __call__(self, batch):
        A, B, C = compute_ABC(batch['w_t_c'], batch['c_R_w'], batch['w_t_chat'], batch['chat_R_w'], self.c_n, self.eye)

        xmin = batch['xmin'].view(-1, 1, 1)
        xmax = batch['xmax'].view(-1, 1, 1)
        B_weight = torch.log(xmax / xmin) / (xmax - xmin)
        C_weight = xmin * xmax

        error = A + B * B_weight + C / C_weight
        error = error.diagonal(dim1=1, dim2=2).sum(dim=1).mean()
        return error



In [3]:
dataset = datasets.SevenScenesDataset(f'/mundus/mrahman527/projects/homography-loss-function/datasets/7-Scenes/fire', 0.025, 0.975)


Loading seq-01


100%|█████████████████████████████████████████████| 1000/1000 [01:00<00:00, 16.49it/s]


Loading seq-02


100%|█████████████████████████████████████████████| 1000/1000 [01:01<00:00, 16.22it/s]


Loading seq-03


100%|█████████████████████████████████████████████| 1000/1000 [01:02<00:00, 16.11it/s]


Loading seq-04


100%|█████████████████████████████████████████████| 1000/1000 [01:00<00:00, 16.47it/s]


Sorting depths, this may take a while...


In [4]:
train_dataset = datasets.RelocDataset(dataset.train_data)
test_dataset = datasets.RelocDataset(dataset.test_data)

trainset_loader = torch.utils.data.DataLoader(train_dataset, shuffle=False, num_workers=6, batch_size=1)
testset_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, num_workers=6, batch_size=1)

#load network
network = Network(torch.zeros((3)),0)

check_path = "our_checkpoints/7-Scenes/fire_with_init"

check_point_list = os.listdir(check_path)


In [5]:

for checkpoint in check_point_list:
    epoch = checkpoint[:-3].split('_')[-1]
    print(f'testing for epoch {epoch}')
    
    check_point = torch.load(os.path.join(check_path, checkpoint))
    
    network.load_state_dict(check_point['model_state_dict'])
    network.cuda()
    network.eval()

    running_homography_loss = 0

    it = 0
    rErrs = []
    tErrs = []
    avg_time = 0
    pct5 = 0
    pct2 = 0
    pct1 = 0

    with torch.no_grad():
        for data in testset_loader:
            it+=1
            print(it)
            focal_length = data['K'][0][0][0]
            image = data['image'].cuda()
            
            wtc, crw = data['w_t_c'], data['c_R_w']

            start_time = time.time()
            scene_coordinates = network(image)
            scene_coordinates = scene_coordinates.cpu()
            gt_pose = reverse_tr(crw, wtc)[0]
            out_pose = torch.zeros((4, 4))
            dsacstar.forward_rgb(
				scene_coordinates, 
				out_pose, 
				64, 
				10,
				focal_length, 
				float(image.size(3) / 2), #principal point assumed in image center
				float(image.size(2) / 2), 
				100,
				100,
				network.OUTPUT_SUBSAMPLE)

            
            avg_time += time.time()-start_time

            # calculate pose errors
            t_err = float(torch.norm(gt_pose[0:3, 3] - out_pose[0:3, 3]))
            # print(f"t err {t_err}")
            gt_R = gt_pose[0:3,0:3].numpy()
            out_R = out_pose[0:3,0:3].numpy()

            r_err = np.matmul(out_R, np.transpose(gt_R))
            r_err = cv2.Rodrigues(r_err)[0]
            r_err = np.linalg.norm(r_err) * 180 / math.pi
            # print(f"r err {r_err}")
            # print(gt_pose)
            # print(out_pose)
    
    median_idx = int(len(rErrs)/2)
    tErrs.sort()
    rErrs.sort()
    avg_time /= len(rErrs)

    print("\n===================================================")
    print("\nTest complete.")

    print('\nAccuracy:')
    print('\n5cm5deg: %.1f%%' %(pct5 / len(rErrs) * 100))
    print('2cm2deg: %.1f%%' % (pct2 / len(rErrs) * 100))
    print('1cm1deg: %.1f%%' % (pct1 / len(rErrs) * 100))

    print("\nMedian Error: %.1fdeg, %.1fcm" % (rErrs[median_idx], tErrs[median_idx]))
    print("Avg. processing time: %4.1fms" % (avg_time * 1000))
    

    break


testing for epoch 0


RuntimeError: CUDA error: an illegal memory access was encountered

In [6]:
!info-cluster


/bin/bash: info-cluster: command not found
